In [2]:
# !wget https://github.com/mesolitica/malaysian-dataset/raw/master/translation/flores200-eval/bjn_Latn.dev
# !wget https://github.com/mesolitica/malaysian-dataset/raw/master/translation/flores200-eval/eng_Latn.dev
# !wget https://github.com/mesolitica/malaysian-dataset/raw/master/translation/flores200-eval/ind_Latn.dev
# !wget https://github.com/mesolitica/malaysian-dataset/raw/master/translation/flores200-eval/jav_Latn.dev
# !wget https://github.com/mesolitica/malaysian-dataset/raw/master/translation/flores200-eval/zsm_Latn.dev
# !wget https://github.com/mesolitica/malaysian-dataset/raw/master/translation/flores200-eval/zho_Hans.dev
# !wget https://github.com/mesolitica/malaysian-dataset/raw/master/translation/flores200-eval/tam_Taml.dev

In [3]:
with open('eng_Latn.dev') as fopen:
    en = fopen.read().split('\n')
    
with open('zsm_Latn.dev') as fopen:
    ms = fopen.read().split('\n')
    
en_, ms_ = [], []
for i in range(len(en)):
    if len(en[i]) and len(ms[i]):
        en_.append(en[i])
        ms_.append(ms[i])

In [4]:
len(en_), len(ms_)

(997, 997)

In [1]:
from tqdm import tqdm
import requests
import os
import json

In [10]:
!mkdir google-translate-ms-en

In [20]:
for i in tqdm(range(len(ms_))):
    filename = os.path.join('google-translate-ms-en', f'{i}.json')
    
    if os.path.exists(filename):
        continue
    
    headers = {
        'accept': 'application/json',
        'Content-Type': 'application/json',
    }

    json_data = {
        'text': ms_[i],
    }

    response = requests.post('http://localhost:8004/translate', headers=headers, json=json_data)
    r = response.json()
    with open(filename, 'w') as fopen:
        json.dump({'text': ms_[i], 'r': r}, fopen)

100%|███████████████████████████████████████████████████████████████████████████████████████████| 997/997 [42:36<00:00,  2.56s/it]


In [5]:
from sacrebleu.metrics import BLEU, CHRF, TER

chrf = CHRF(word_order = 2)

In [21]:
predicted, actual = [], []
for i in range(len(ms_)):
    try:
        filename = os.path.join('google-translate-ms-en', f'{i}.json')
        with open(filename) as fopen:
            d = json.load(fopen)
            
        n = d['r']['result']
        predicted.append(d['r']['result'])
        actual.append(en_[i])
    except:
        pass

In [23]:
score = chrf.corpus_score(predicted, [actual])
score

chrF2++ = 60.19